In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
file_path = r"C:\Users\alitr\Downloads\Moonlight Solution\Moonlights-Energy-Solutions\benin-malanville.csv"
df = pd.read_csv(file_path)


In [ ]:
import pandas as pd

# File path (choose one of the above methods)
file_path = r"C:\Users\alitr\OneDrive\Desktop\data\benin-malanville.csv"

# Load the dataset
df = pd.read_csv(file_path)

# Display the first few rows
df.head()


In [ ]:
# Importing necessary libraries
import pandas as pd

# Identify numeric columns
numeric_columns = [
    'GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'Tamb', 'RH', 'WS', 
    'WSgust', 'WSstdev', 'WD', 'WDstdev', 'BP', 'TModA', 'TModB'
]

# Calculate summary statistics for selected numeric columns
summary_stats = df[numeric_columns].describe().T

# Adding median and variance to the summary
summary_stats['median'] = df[numeric_columns].median()
summary_stats['variance'] = df[numeric_columns].var()

# Formatting the result into a table-like structure
summary_stats = summary_stats[['count', 'mean', 'std', 'min', 'max' ,'median', 'variance']]

# Displaying the summary statistics table
summary_stats


In [ ]:
import pandas as pd

# Assuming your DataFrame is named 'df'
# Check for missing values in the specific columns: 'GHI', 'DNI', 'DHI'
missing_values_selected = df[['GHI', 'DNI', 'DHI']].isnull().sum()

# Display missing values for the selected columns
print("Missing Values in GHI, DNI, and DHI:")
print(missing_values_selected)


In [ ]:
# Calculate the IQR for ModA, ModB, WS, WSgust columns
Q1_sensor = df[['ModA', 'ModB', 'WS', 'WSgust']].quantile(0.25)
Q3_sensor = df[['ModA', 'ModB', 'WS', 'WSgust']].quantile(0.75)
IQR_sensor = Q3_sensor - Q1_sensor

# Define the outlier condition: values below Q1 - 1.5*IQR or above Q3 + 1.5*IQR
outliers_sensor = ((df[['ModA', 'ModB', 'WS', 'WSgust']] < (Q1_sensor - 1.5 * IQR_sensor)) | 
                   (df[['ModA', 'ModB', 'WS', 'WSgust']] > (Q3_sensor + 1.5 * IQR_sensor)))

# Display the outliers count for each column
outliers_count = outliers_sensor.sum()
print("\nOutliers Count in ModA, ModB, WS, and WSgust:")
print(outliers_count)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Step 1: Set 'Timestamp' as the index and ensure it's in datetime format
df.set_index('Timestamp', inplace=True)
df.index = pd.to_datetime(df.index)

# Step 2: Resample by month, calculating the mean for each month
df_monthly = df.resample('M').mean()

# Step 3: Resample by day, calculating the mean for each day
df_daily = df.resample('D').mean()

# Step 4: Plot the resampled data for both monthly and daily averages
fig, axes = plt.subplots(2, 1, figsize=(12, 12))

# Plot for monthly data
df_monthly[['GHI', 'DNI', 'DHI', 'Tamb']].plot(ax=axes[0], kind='line', title='Monthly Average of GHI, DNI, DHI, and Tamb')
axes[0].set_ylabel('Value')
axes[0].set_xlabel('Month')
axes[0].legend(title='Variables')

# Plot for daily data
df_daily[['GHI', 'DNI', 'DHI', 'Tamb']].plot(ax=axes[1], kind='line', title='Daily Average of GHI, DNI, DHI, and Tamb')
axes[1].set_ylabel('Value')
axes[1].set_xlabel('Day')
axes[1].legend(title='Variables')

# Show the plots
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Step 1: Compute the correlation matrix for solar radiation components and temperature measures
correlation_data = df[['GHI', 'DNI', 'DHI', 'TModA', 'TModB']]

# Compute the correlation matrix
correlation_matrix = correlation_data.corr()

# Step 2: Visualize the correlation matrix using a heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Correlation Matrix for Solar Radiation and Temperature Measures')
plt.show()

# Step 3: Use pair plots to visualize correlations between solar radiation and temperature
sns.pairplot(correlation_data, kind='scatter')
plt.suptitle('Pair Plot of Solar Radiation and Temperature Measures', y=1.02)
plt.show()

# Step 4: Investigate the relationship between wind conditions and solar irradiance
# We'll plot the scatter matrix for the wind speed (WS, WSgust, WD) vs. solar radiation components (GHI, DNI, DHI)
wind_solar_data = df[['WS', 'WSgust', 'WD', 'GHI', 'DNI', 'DHI']]

# Step 5: Visualize the relationships using a scatter matrix (pairplot)
sns.pairplot(wind_solar_data, kind='scatter')
plt.suptitle('Scatter Matrix for Wind Conditions and Solar Irradiance', y=1.02)
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Step 1: Create a Wind Rose
from windrose import WindroseAxes

# Prepare the data for the wind rose (wind speed and wind direction)
wind_data = df[['WS', 'WD']]  # Assuming 'WS' is wind speed and 'WD' is wind direction

# Step 2: Plot the Wind Rose
fig = plt.figure(figsize=(8, 6))
ax = WindroseAxes.from_ax()
ax.bar(wind_data['WD'], wind_data['WS'], bins=16, edgecolor='black', alpha=0.7)
ax.set_title('Wind Rose: Wind Speed Distribution by Direction')
plt.show()

# Step 3: Radial Bar Plot for Wind Speed by Direction
# We'll create a radial bar plot that shows the distribution of wind speeds
# Convert wind direction to radians for the radial plot
wind_data['WD_rad'] = np.deg2rad(wind_data['WD'])

# Number of bins to categorize wind speed into groups (you can adjust as needed)
wind_speed_bins = np.linspace(0, wind_data['WS'].max(), 6)  # Example with 5 wind speed bins

# Create a new column 'Wind Speed Bin' to categorize the wind speed
wind_data['Wind Speed Bin'] = pd.cut(wind_data['WS'], bins=wind_speed_bins, labels=['0-5', '6-10', '11-15', '16-20', '21+'])

# Calculate the mean wind speed for each direction bin
mean_wind_speed = wind_data.groupby('WD_rad')['WS'].mean()

# Plotting the radial bar plot
fig, ax = plt.subplots(figsize=(8, 8), dpi=100, subplot_kw={'projection': 'polar'})
bars = ax.bar(wind_data['WD_rad'].unique(), mean_wind_speed, width=0.3, color='skyblue', edgecolor='black', alpha=0.7)

ax.set_title('Radial Bar Plot: Mean Wind Speed by Direction')
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Step 1: Calculate correlations between variables
correlation_matrix = df[['RH', 'ModA', 'ModB', 'GHI', 'DNI', 'DHI']].corr()

# Step 2: Plot a heatmap of the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title("Correlation Heatmap: RH, Temperature (ModA, ModB), and Solar Radiation (GHI, DNI, DHI)")
plt.show()

# Step 3: Scatter Plots to visualize relationships
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# RH vs ModA
sns.scatterplot(x='RH', y='ModA', data=df, ax=axes[0, 0], color='blue')
axes[0, 0].set_title('RH vs ModA')

# RH vs ModB
sns.scatterplot(x='RH', y='ModB', data=df, ax=axes[0, 1], color='green')
axes[0, 1].set_title('RH vs ModB')

# RH vs GHI
sns.scatterplot(x='RH', y='GHI', data=df, ax=axes[0, 2], color='red')
axes[0, 2].set_title('RH vs GHI')

# RH vs DNI
sns.scatterplot(x='RH', y='DNI', data=df, ax=axes[1, 0], color='purple')
axes[1, 0].set_title('RH vs DNI')

# RH vs DHI
sns.scatterplot(x='RH', y='DHI', data=df, ax=axes[1, 1], color='orange')
axes[1, 1].set_title('RH vs DHI')

# RH vs ModA and ModB (combined plot)
sns.scatterplot(x='RH', y='ModA', data=df, ax=axes[1, 2], color='blue', label='ModA')
sns.scatterplot(x='RH', y='ModB', data=df, ax=axes[1, 2], color='green', label='ModB')
axes[1, 2].set_title('RH vs ModA & ModB')
axes[1, 2].legend()

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# List of variables to plot histograms for
variables = ['GHI', 'DNI', 'DHI', 'WS', 'ModA', 'ModB']

# Set up the matplotlib figure
plt.figure(figsize=(15, 10))

# Loop through the variables and plot histograms
for i, var in enumerate(variables):
    plt.subplot(2, 3, i+1)  # 2 rows, 3 columns
    sns.histplot(df[var], kde=True, bins=20, color='blue', edgecolor='black')
    plt.title(f"Histogram of {var}")
    plt.xlabel(var)
    plt.ylabel("Frequency")

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import numpy as np

# Assuming your dataframe is named df
# List of columns you want to calculate Z-scores for
columns_to_check = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'Tamb', 'RH', 'WS', 'WSgust', 'BP']

# Calculate Z-scores for the selected columns
z_scores = (df[columns_to_check] - df[columns_to_check].mean()) / df[columns_to_check].std()

# Flagging outliers - assuming a Z-score threshold of 3 or -3
outliers = (z_scores > 3) | (z_scores < -3)

# Add a new column 'Outlier_Flag' to indicate if a data point is an outlier (True) or not (False)
df['Outlier_Flag'] = outliers.any(axis=1)

# Display the rows that contain outliers
outliers_df = df[df['Outlier_Flag'] == True]

# Print the outliers if any
print(outliers_df)


In [ ]:
import pandas as pd
import numpy as np

# Assuming your DataFrame is named df
# List of columns to calculate Z-scores for
columns_to_check = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'Tamb', 'RH', 'WS', 'WSgust', 'BP']

# Calculate Z-scores for the selected columns
z_scores = (df[columns_to_check] - df[columns_to_check].mean()) / df[columns_to_check].std()

# Flagging outliers - flag Z-scores greater than 3 or less than -3
outliers = (z_scores > 3) | (z_scores < -3)

# Add a new column 'Outlier_Flag' to indicate if a data point is an outlier (True) or not (False)
df['Outlier_Flag'] = outliers.any(axis=1)

# Display rows that contain outliers
outliers_df = df[df['Outlier_Flag'] == True]

# Print the outliers if any
print(outliers_df)


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Load your dataset into a DataFrame (using the correct file path)
file_path = r"C:\Users\alitr\OneDrive\Desktop\data\benin-malanville.csv"
df = pd.read_csv(file_path, parse_dates=True, index_col='Timestamp')

# Check if the necessary columns exist in the dataset
required_columns = ['GHI', 'Tamb', 'RH', 'WS']
if all(col in df.columns for col in required_columns):
    # X-axis: GHI (Global Horizontal Irradiance)
    # Y-axis: Tamb (Temperature)
    # Bubble size: RH (Relative Humidity or BP (Barometric Pressure))
    # Bubble color: WS (Wind Speed)
    
    x = df['GHI']   # X-axis variable (GHI)
    y = df['Tamb']  # Y-axis variable (Temperature)
    size = df['RH'] # Size of the bubble (Relative Humidity or Barometric Pressure)
    color = df['WS'] # Color of the bubble (Wind Speed)

    # Create the bubble chart
    plt.figure(figsize=(10, 6))
    scatter = plt.scatter(x, y, s=size*5, c=color, cmap='viridis', alpha=0.6, edgecolors="w", linewidth=0.5)

    # Adding labels and title
    plt.title('Bubble Chart: GHI vs. Tamb vs. WS', fontsize=16)
    plt.xlabel('Global Horizontal Irradiance (GHI)', fontsize=12)
    plt.ylabel('Temperature (Tamb)', fontsize=12)

    # Add a color bar to show Wind Speed values
    plt.colorbar(scatter, label='Wind Speed (WS)')

    # Show the plot
    plt.show()
else:
    print("Required columns not found in the dataset.")


In [ ]:
import pandas as pd

# Load your dataset into a DataFrame
file_path = r"C:\Users\alitr\OneDrive\Desktop\data\benin-malanville.csv"
df = pd.read_csv(file_path, parse_dates=True, index_col='Timestamp')

# 1. Check for missing values
print("Missing values per column:")
print(df.isnull().sum())

# 2. Handle missing values:
# - Drop the 'Comments' column since it appears entirely null
# - Alternatively, fill missing columns with appropriate methods (e.g., fill with mean or mode)

# Dropping 'Comments' column
df.drop(columns=['Comments'], inplace=True)

# Example: Fill missing numeric values with the mean (for columns like GHI, Tamb, RH, WS)
df.fillna(df.mean(), inplace=True)

# Example: If there are categorical columns with missing values (fill with mode)
# df['CategoryColumn'] = df['CategoryColumn'].fillna(df['CategoryColumn'].mode()[0])

# 3. Handle duplicate rows if any
df.drop_duplicates(inplace=True)

# 4. Check for anomalies (e.g., negative values where they don't make sense, extreme outliers)
# For instance, GHI (Global Horizontal Irradiance) should not be negative
df = df[df['GHI'] >= 0]

# You can also perform more detailed anomaly detection, depending on the column's nature.

# 5. Verify the changes
print("\nDataset after cleaning:")
print(df.info())

# Optionally, save the cleaned dataset to a new file
# df.to_csv("cleaned_benin_malanville.csv", index=False)
